A little notebook to help visualise the official numbers for personal use. Absolutely no guarantees are made.

**This is not a replacement for expert advice. Please listen to your local health authorities.**

The data is dynamically loaded from: https://github.com/CSSEGISandData/COVID-19

Maybe switch to: https://data.humdata.org/dataset/5dff64bc-a671-48da-aa87-2ca40d7abf02

In [1]:
%matplotlib ipympl
#%matplotlib inline
#%config InlineBackend.figure_format ='retina'

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from jhu_helpers import *

In [3]:
jhu = aggregte_jhu_by_state(*get_jhu_data())

In [4]:
#jhu.confirmed.columns.tolist() # print a list of all countries in the data set

In [5]:
# look at recent numbers from highly affected countries
get_aggregate_top_n(jhu.confirmed)

Country/Region,US,Spain,Italy,France,Germany,United Kingdom,China,Iran,Turkey,Belgium,Netherlands,Canada,Switzerland,Brazil,Russia,Portugal,Austria,Israel,India
Date,,,,,,,,,,,,,,,,,,,
2020-04-10,496535,158273,147577,125931,122171,74605,82941,68192,47029,26667,23249,22059,24551,19638,11917,15472,13555,10408,7598
2020-04-11,526396,163027,152271,130727,124908,79874,83014,70029,52167,28018,24571,23316,25107,20727,13584,15987,13806,10743,8446
2020-04-12,555313,166831,156363,133670,127854,85206,83134,71686,56956,29647,25746,24298,25415,22192,15770,16585,13945,11145,9205
2020-04-13,580619,170099,159516,137875,130072,89570,83213,73303,61049,30589,26710,25679,25688,23430,18328,16934,14041,11586,10453
2020-04-14,607670,172541,162488,131361,131359,94845,83306,74877,65111,31119,27580,27034,25936,25262,21102,17448,14226,12046,11487


In [6]:
# choose a random list of countries to plot
plot_countries = [
    'China',
    'Italy',
    #'Singapore',
    'Korea, South',
    'US',
    'France',
    'Germany',
]

In [7]:
plt.close(1)
fig1, ax1 = plt.subplots(nrows=2, ncols=2, figsize=(10,8), sharex=True, num=1)

jhu.confirmed[plot_countries].plot(ax=ax1[0,0], logy=True)
ax1[0,0].set_ylabel('Confirmed Cases')

jhu.deaths[plot_countries].plot(ax=ax1[0,1], logy=True)
ax1[0,1].set_ylabel('Deaths')

jhu.new_infected_21d[plot_countries].plot(ax=ax1[1,0], logy=True)
ax1[1,0].set_ylabel('New Confirmations in 21 days')

jhu.new_infection_rate_21d[plot_countries].plot(ax=ax1[1,1], logy=False)
ax1[1,1].set_ylabel('21-day Confirmation Growth Rate')

fig1.tight_layout()

FigureCanvasNbAgg()

In [8]:
# save the above figure
#fig1.savefig('sars-covid-19_timeseries.png')

### Calculate days during which confirmed cases doubled

In [9]:
# confirmed 2d-array with last date in first row
confirmed = jhu.confirmed.reindex(
    # make sure we have one row for each day
    pd.date_range(jhu.index[0], jhu.index[-1], freq='1d'),
    method='ffill',
    axis=0
).loc[::-1].values

days_since_doubling = np.empty(confirmed.shape)
for i in range(len(confirmed)):
    days_since_doubling[i] = (confirmed <= confirmed[i] / 2).argmax(axis=0) - i

days_since_doubling[days_since_doubling < 0] = np.nan

days_since_doubling = pd.DataFrame(
    days_since_doubling, 
    index=jhu.index[::-1], 
    columns=jhu.confirmed.columns
).sort_index()

days_since_doubling.tail()

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-10,9.0,13.0,9.0,14.0,7.0,8.0,11.0,12.0,15.0,16.0,...,8.0,15.0,6.0,17.0,18.0,10.0,6.0,1.0,15.0,14.0
2020-04-11,9.0,13.0,10.0,15.0,8.0,9.0,12.0,12.0,15.0,17.0,...,8.0,15.0,6.0,18.0,19.0,10.0,7.0,2.0,16.0,12.0
2020-04-12,8.0,13.0,11.0,15.0,9.0,10.0,11.0,13.0,16.0,17.0,...,8.0,16.0,7.0,19.0,20.0,11.0,8.0,3.0,17.0,13.0
2020-04-13,9.0,14.0,11.0,16.0,10.0,11.0,12.0,14.0,17.0,18.0,...,9.0,17.0,7.0,19.0,21.0,12.0,9.0,4.0,17.0,12.0
2020-04-14,9.0,15.0,12.0,17.0,11.0,12.0,12.0,14.0,18.0,19.0,...,10.0,18.0,5.0,20.0,22.0,13.0,10.0,5.0,18.0,13.0


In [10]:
plt.close(2)
fig2, ax2 = plt.subplots(figsize=(6,5), sharex=True, num=2)

days_since_doubling[plot_countries].plot(ax=ax2, logy=False)
ax2.set_ylim([1,30])
ax2.grid(axis='y')
ax2.set_ylabel('Days it took for last doubling of confirmed cases')
fig2.tight_layout()

FigureCanvasNbAgg()

In [11]:
#fig2.savefig('doubling.png', dpi=300)